In [ ]:
!unzip -q "/content/drive/MyDrive/fraud.zip" -d "/content/drive/MyDrive/fraud"


In [ ]:
!pip uninstall -y keras keras-nightly keras-preprocessing keras-vis keras-applications
!pip install tensorflow==2.15.0 keras==2.15.0


Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf

holo

In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from tqdm import tqdm

# Paramètres
patch_size = 28
model_input_size = 224
image_size = (595, 412)  # largeur, hauteur
seuil_prediction = 0.4
alpha = 0.5  # transparence pour superposition

# Chemins
model_path = '/content/drive/MyDrive/mobilenet_custom_trained2/best_model.h5'
mask_path = '/content/drive/MyDrive/passport_hologram_mask_resized_412x595.png'
video_input_dir = '/content/drive/MyDrive/Archive/videos/images/origins/passport_cropped'
video_output_dir = '/content/drive/MyDrive/videos2_holo'

os.makedirs(video_output_dir, exist_ok=True)

print(f"Chargement du modèle depuis : {model_path}")
model = load_model(model_path)
print("Modèle chargé.")

# Chargement et traitement masque
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask = cv2.resize(mask, image_size)
_, mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)

def extract_patches(img):
    patches, positions = [], []
    for y in range(0, img.shape[0] - patch_size + 1, patch_size):
        for x in range(0, img.shape[1] - patch_size + 1, patch_size):
            patches.append(img[y:y+patch_size, x:x+patch_size])
            positions.append((x, y))
    return patches, positions

def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Erreur: impossible d'ouvrir la vidéo {video_path}")
        return

    width, height = image_size
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    print(f"Traitement de la vidéo {os.path.basename(video_path)} avec {frame_count} frames.")

    # Initialisation accumulation prédictions
    first_frame = True
    prediction_sum = None
    prediction_count = 0

    for _ in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensionner la frame à la taille attendue
        frame_resized = cv2.resize(frame, (width, height))

        # Extraire patches et positions
        patches, positions = extract_patches(frame_resized)

        if first_frame:
            num_patches = len(patches)
            prediction_sum = np.zeros(num_patches, dtype=np.float32)
            patch_positions = positions
            first_frame = False

        # Préparation des patches pour le modèle
        batch_input = [cv2.resize(p, (model_input_size, model_input_size)) for p in patches]
        batch_input = np.array(batch_input).astype(np.float32) / 255.0

        # Prédiction
        predictions = model.predict(batch_input, verbose=0).flatten()
        prediction_sum += predictions
        prediction_count += 1

    cap.release()

    # Moyenne des prédictions
    prediction_mean = prediction_sum / prediction_count

    # Génération de l'image couleur finale (fixe)
    coloration_image = np.zeros((height, width, 3), dtype=np.uint8)

    for pred, (x, y) in zip(prediction_mean, patch_positions):
        mask_patch = mask[y:y+patch_size, x:x+patch_size]
        in_holo_zone = (mask_patch == 1).any()
        is_holo = (pred >= seuil_prediction) and in_holo_zone
        color = (0, 255, 0) if is_holo else (0, 0, 255)  # vert ou rouge
        coloration_image[y:y+patch_size, x:x+patch_size] = color

    # Réouvrir la vidéo pour écrire la sortie
    cap = cv2.VideoCapture(video_path)

    for _ in tqdm(range(frame_count), desc="Écriture de la vidéo"):
        ret, frame = cap.read()
        if not ret:
            break
        frame_resized = cv2.resize(frame, (width, height))
        output_frame = cv2.addWeighted(frame_resized, 1 - alpha, coloration_image, alpha, 0)
        out.write(output_frame)

    cap.release()
    out.release()
    print(f"Vidéo sauvegardée dans : {output_path}")

# Traitement batch
for file in os.listdir(video_input_dir):
    if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        input_video_path = os.path.join(video_input_dir, file)
        output_video_path = os.path.join(video_output_dir, f"{os.path.splitext(file)[0]}_colored_mean_fixed.mp4")
        process_video(input_video_path, output_video_path)

print("Traitement terminé pour toutes les vidéos.")


Chargement du modèle depuis : /content/drive/MyDrive/mobilenet_custom_trained2/best_model.h5
Modèle chargé.
Traitement de la vidéo passport_psp01_03_03.mp4 avec 40 frames.


Écriture de la vidéo: 100%|██████████| 40/40 [00:00<00:00, 210.11it/s]


Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_holo/passport_psp01_03_03_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp02_04_02.mp4 avec 46 frames.


Écriture de la vidéo: 100%|██████████| 46/46 [00:00<00:00, 216.62it/s]


Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_holo/passport_psp02_04_02_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp07_01_01.mp4 avec 49 frames.


...

Écriture de la vidéo: 100%|██████████| 43/43 [00:00<00:00, 209.10it/s]


Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_holo/passport_psp03_02_03_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp07_01_02.mp4 avec 46 frames.


Écriture de la vidéo: 100%|██████████| 46/46 [00:00<00:00, 165.41it/s]


Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_holo/passport_psp07_01_02_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp07_01_03.mp4 avec 44 frames.


 89%|████████▊ | 39/44 [06:37<00:52, 10.50s/it]

avec no holo copy_without_holo

In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from tqdm import tqdm

# Paramètres
patch_size = 28
model_input_size = 224
image_size = (595, 412)  # largeur, hauteur
seuil_prediction = 0.4
alpha = 0.5  # transparence pour superposition

# Chemins
model_path = '/content/drive/MyDrive/mobilenet_custom_trained2/best_model.h5'
video_input_dir = '/content/drive/MyDrive/fraud/fraud/copy_without_holo/passport_cropped'
video_output_dir = '/content/drive/MyDrive/videos2_noholo'

os.makedirs(video_output_dir, exist_ok=True)

print(f"Chargement du modèle depuis : {model_path}")
model = load_model(model_path)
print("Modèle chargé.")

def extract_patches(img):
    patches, positions = [], []
    for y in range(0, img.shape[0] - patch_size + 1, patch_size):
        for x in range(0, img.shape[1] - patch_size + 1, patch_size):
            patches.append(img[y:y+patch_size, x:x+patch_size])
            positions.append((x, y))
    return patches, positions

def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Erreur: impossible d'ouvrir la vidéo {video_path}")
        return

    width, height = image_size
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    print(f"Traitement de la vidéo {os.path.basename(video_path)} avec {frame_count} frames.")

    # Initialisation accumulation prédictions
    first_frame = True
    prediction_sum = None
    prediction_count = 0

    for _ in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensionner la frame à la taille attendue
        frame_resized = cv2.resize(frame, (width, height))

        # Extraire patches et positions
        patches, positions = extract_patches(frame_resized)

        if first_frame:
            num_patches = len(patches)
            prediction_sum = np.zeros(num_patches, dtype=np.float32)
            patch_positions = positions
            first_frame = False

        # Préparation des patches pour le modèle
        batch_input = [cv2.resize(p, (model_input_size, model_input_size)) for p in patches]
        batch_input = np.array(batch_input).astype(np.float32) / 255.0

        # Prédiction
        predictions = model.predict(batch_input, verbose=0).flatten()
        prediction_sum += predictions
        prediction_count += 1

    cap.release()

    prediction_mean = prediction_sum / prediction_count

    # Génération de l'image couleur finale (fixe)
    coloration_image = np.zeros((height, width, 3), dtype=np.uint8)

    for pred, (x, y) in zip(prediction_mean, patch_positions):
        is_holo = pred >= seuil_prediction
        color = (0, 255, 0) if is_holo else (0, 0, 255)
        coloration_image[y:y+patch_size, x:x+patch_size] = color

    # Réouvrir la vidéo pour écrire la sortie
    cap = cv2.VideoCapture(video_path)

    for _ in tqdm(range(frame_count), desc="Écriture de la vidéo"):
        ret, frame = cap.read()
        if not ret:
            break
        frame_resized = cv2.resize(frame, (width, height))
        output_frame = cv2.addWeighted(frame_resized, 1 - alpha, coloration_image, alpha, 0)
        out.write(output_frame)

    cap.release()
    out.release()
    print(f"Vidéo sauvegardée dans : {output_path}")

# Traitement batch
for file in os.listdir(video_input_dir):
    if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        input_video_path = os.path.join(video_input_dir, file)
        output_video_path = os.path.join(video_output_dir, f"{os.path.splitext(file)[0]}_colored_mean_fixed.mp4")
        process_video(input_video_path, output_video_path)

print("Traitement terminé pour toutes les vidéos.")


Chargement du modèle depuis : /content/drive/MyDrive/mobilenet_custom_trained2/best_model.h5
Modèle chargé.
Traitement de la vidéo passport_psp07_01_01.mp4 avec 41 frames.


Écriture de la vidéo: 100%|██████████| 41/41 [00:00<00:00, 270.73it/s]


Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_noholo/passport_psp07_01_01_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp03_02_01.mp4 avec 39 frames.


Écriture de la vidéo: 100%|██████████| 39/39 [00:00<00:00, 274.39it/s]


Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_noholo/passport_psp03_02_01_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp02_04_01.mp4 avec 42 frames.


...

Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_noholo/passport_psp08_02_01_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp04_04_01.mp4 avec 40 frames.


Écriture de la vidéo: 100%|██████████| 40/40 [00:00<00:00, 156.31it/s]


Vidéo sauvegardée dans : /content/drive/MyDrive/videos2_noholo/passport_psp04_04_01_colored_mean_fixed.mp4
Traitement de la vidéo passport_psp10_04_01.mp4 avec 40 frames.


  0%|          | 0/40 [00:03<?, ?it/s]


KeyboardInterrupt: 

tester pseudo holo copy

In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from tqdm import tqdm

# Paramètres
patch_size = 28
model_input_size = 224
image_size = (595, 412)  # largeur, hauteur
seuil_prediction = 0.7
alpha = 0.5  # transparence pour superposition

# Chemins
model_path = '/content/drive/MyDrive/mobilenet_custom_trained2/best_model.h5'
mask_path = '/content/drive/MyDrive/passport_hologram_mask_resized_412x595.png'
video_input_dir = '/content/drive/MyDrive/fraud/fraud/pseudo_holo_copy/passport_cropped'
video_output_dir = '/content/drive/MyDrive/videos3_noholo'

os.makedirs(video_output_dir, exist_ok=True)

print(f"Chargement du modèle depuis : {model_path}")
model = load_model(model_path)
print("Modèle chargé.")

# Chargement et traitement masque
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask = cv2.resize(mask, image_size)
_, mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)

def extract_patches(img):
    patches, positions = [], []
    for y in range(0, img.shape[0] - patch_size + 1, patch_size):
        for x in range(0, img.shape[1] - patch_size + 1, patch_size):
            patches.append(img[y:y+patch_size, x:x+patch_size])
            positions.append((x, y))
    return patches, positions

def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Erreur: impossible d'ouvrir la vidéo {video_path}")
        return

    width, height = image_size
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    print(f"Traitement de la vidéo {os.path.basename(video_path)} avec {frame_count} frames.")

    # Initialisation accumulation prédictions
    first_frame = True
    prediction_sum = None
    prediction_count = 0

    for _ in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensionner la frame à la taille attendue
        frame_resized = cv2.resize(frame, (width, height))

        # Extraire patches et positions
        patches, positions = extract_patches(frame_resized)

        if first_frame:
            num_patches = len(patches)
            prediction_sum = np.zeros(num_patches, dtype=np.float32)
            patch_positions = positions
            first_frame = False

        # Préparation des patches pour le modèle
        batch_input = [cv2.resize(p, (model_input_size, model_input_size)) for p in patches]
        batch_input = np.array(batch_input).astype(np.float32) / 255.0

        # Prédiction
        predictions = model.predict(batch_input, verbose=0).flatten()
        prediction_sum += predictions
        prediction_count += 1

    cap.release()

    # Moyenne des prédictions
    prediction_mean = prediction_sum / prediction_count

    # Génération de l'image couleur finale (fixe)
    coloration_image = np.zeros((height, width, 3), dtype=np.uint8)

    for pred, (x, y) in zip(prediction_mean, patch_positions):
        mask_patch = mask[y:y+patch_size, x:x+patch_size]
        in_holo_zone = (mask_patch == 1).any()
        is_holo = (pred >= seuil_prediction) and in_holo_zone
        color = (0, 255, 0) if is_holo else (0, 0, 255)  # vert ou rouge
        coloration_image[y:y+patch_size, x:x+patch_size] = color

    # Réouvrir la vidéo pour écrire la sortie
    cap = cv2.VideoCapture(video_path)

    for _ in tqdm(range(frame_count), desc="Écriture de la vidéo"):
        ret, frame = cap.read()
        if not ret:
            break
        frame_resized = cv2.resize(frame, (width, height))
        output_frame = cv2.addWeighted(frame_resized, 1 - alpha, coloration_image, alpha, 0)
        out.write(output_frame)

    cap.release()
    out.release()
    print(f"Vidéo sauvegardée dans : {output_path}")

# Traitement batch
for file in os.listdir(video_input_dir):
    if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        input_video_path = os.path.join(video_input_dir, file)
        output_video_path = os.path.join(video_output_dir, f"{os.path.splitext(file)[0]}_colored_mean_fixed.mp4")
        process_video(input_video_path, output_video_path)

print("Traitement terminé pour toutes les vidéos.")


KeyboardInterrupt: 